In [16]:
import pandas as pd
from abc import ABC
from gensim.corpora import Dictionary
from torch.utils.data import Dataset
import gensim.downloader as api
from time import time
import gensim
import gensim.downloader as api


from gensim.models import Word2Vec
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
import pandas as pd
from torch.utils.data import DataLoader
from collections import Counter

import torch.nn as nn
import torch.nn.functional as F

def load_data(name):
    df = pd.read_csv(f'./data/{name}.csv')
    df.is_fulltime = df.is_fulltime.astype(str)
    train = pd.read_csv(f'./data/{name}-train.csv')
    train.is_fulltime = train.is_fulltime.astype(str)
    test = pd.read_csv(f'./data/{name}-test.csv')
    test.is_fulltime = test.is_fulltime.astype(str)
    val = pd.read_csv(f'./data/{name}-val.csv')
    val.is_fulltime = val.is_fulltime.astype(str)
    return df, train, test, val

In [2]:
class Word2VecVectoriser(object):
    def __init__(self, df, load_path=None):
        self.job_descriptions = df.job_description
        if not load_path:
            self.model = self.model_base()
            self.build_vocab()
            self.train_model()
        else:
            self.model = gensim.models.KeyedVectors.load(load_path)
    
    def model_base(self):
        cores = 4
        model = Word2Vec(min_count=1,
                         window=2,
                         vector_size=100,
                         sample=6e-5, 
                         alpha=0.03, 
                         min_alpha=0.0007, 
                         negative=20,
                         workers=cores-1)
        return model

    def build_vocab(self):
        t = time()
        self.model.build_vocab(self.job_descriptions, progress_per=10)
        print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

    def train_model(self):
        t = time()
        self.model.train(self.job_descriptions, total_examples=self.model.corpus_count, epochs=50, report_delay=1)
        print('Time to train : {} mins'.format(round((time() - t) / 60, 2)))
        
    def get_vectoriser(self):
        return self.model
    
    def vectorise(self, words):
        indicies = [self.model.wv.get_index(word) for word in words]
        return self.model.wv[indicies]
    

In [38]:
task1 = pd.read_csv(f'./data/task1.csv')

In [4]:
# class OneHotVectoriser(object):
#     def __init__(self, corpus):
#         self.dictionary = Dictionary(corpus)
#         self.vectoriser = self.get_one_hot_vectoriser()
    
#     def get_one_hot_vectoriser(self):
#         def one_hot_vectoriser(word):
#             indx = self.dictionary.token2id[word]
#             # could throw an exception here I guess
#             vec = np.zeros(2928)
#             vec[indx] = 1
#             return vec
    
#         return one_hot_vectoriser
    
#     def get_vectoriser(self):
#         return self.vecotoriser
    
#     def vectorise(self, words):
#         return [self.vectoriser.one_hot_vectoriser(word) for word in words]
    

In [5]:
# class PretrainedVectoriser(object):
#     def __init__(self, path=None):
#         self.vectoriser = self.create_pretrained(load_path=path)
    
#     def create_pretrained(self, load_path=None):
#         if not load_path:
#             pretrained_w2v = api.load('word2vec-google-news-300')
#             return pretrained_w2v
#         else:
#             return gensim.models.KeyedVectors.load(load_path)
         
#     def get_vectoriser(self):
#         return self.vecotoriser

#     def vectorise(self, words):
#         indicies = [self.vecotoriser.get_index(word) for word in words]
#         return self.vectoriser[indicies]
    

In [49]:
class JobDescriptionDataset(Dataset):
    def __init__(self, df, train, val, test, vectorizer):
        """
        Args:
        review_df (pandas.DataFrame): the dataset
        vectorizer (Vectorizer): vectorizer instantiated from dataset
        """
        self.df = df
        self.train = train
        self.val = val 
        self.test = test 
        self._vectorizer = vectorizer
        self.train_size = len(self.train)
        self.validation_size = len(self.val)
        self.test_size = len(self.test)
        self._lookup_dict = {'train': (self.train, self.train_size),
                            'val': (self.val, self.validation_size),
                            'test': (self.test, self.test_size)}
        self.set_split('train')
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, name):
        """Load dataset and make a new vectorizer from scratch
        Args:
             name (str): name of task to load
        Returns:
            an instance of ReviewDataset
        """
        df, train, test, val = load_data(name)
        return cls(df, train, test, val, OneHotVectoriser.from_dataframe(df)) 
    
    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer
    
    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe
        Args:
        split (str): one of "train", "val", or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
        
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        Args:
        index (int): the index to the data point
        Returns:
        a dict of the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]
        input_vector = \
        self._vectorizer.vectorize(row.tfidf10)
        class_index = \
        self._vectorizer.class_vocab.lookup_token(row.is_fulltime)
        return {'x_data': input_vector,
                'y_target': class_index}
    
    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        Args:
        batch_size (int)
        Returns:
        number of batches in the dataset
        """
        return len(self) // batch_size

In [50]:
class Vocabulary(object):
    """Class to process text and extract Vocabulary for mapping"""
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        Args:
            token_to_idx (dict): a pre-existingmap of tokens to indices
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary
        """
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token 
                                for token, idx in self._token_to_idx.items()}
        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = 1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
            
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}
    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)
    
    def add_token(self, token):
        """Update mapping dicts based on the token.
        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def lookup_token(self, token):
        """Retrieve the index associated with the token
        or the UNK index if token isn't present.
        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
            for the UNK functionality
        """
        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
        
    def lookup_index(self, index):
        """Return the token associated with the index
        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
        KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    def __len__(self):
        return len(self._token_to_idx)

In [51]:
class OneHotVectoriser(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, input_vocab, class_vocab):
        """
        Args:
            review_vocab (Vocabulary): maps words to integers
            rating_vocab (Vocabulary): maps class labels to integers
        """
        self.input_vocab = input_vocab
        self.class_vocab = class_vocab
        
    def vectorize(self, words):
        """Create a collapsed one hot vector for the review
        Args:
            review (str): the review
        Returns:
            one_hot (np.ndarray): the collapsed onehot encoding
        """
        one_hot = np.zeros(len(self.input_vocab), dtype=np.float32)
        for token in words:
            one_hot[self.input_vocab.lookup_token(token)] = 1
        return one_hot
    @classmethod
    def from_dataframe(cls, df, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        Args:
            review_df (pandas.DataFrame): the review dataset
            cutoff (int): the parameter for frequency based filtering
        Returns:
            an instance of the ReviewVectorizer
        """
        input_vocab = Vocabulary(add_unk=True)
        class_vocab = Vocabulary(add_unk=False)
        # Add ratings
        for cl in sorted(set(df.is_fulltime)):
            class_vocab.add_token(cl)
        # Add top words if count > provided count
        word_counts = Counter()
        for wordlist in df.tfidf10:
            for word in wordlist:
                word_counts[word] += 1
        for word, count in word_counts.items():
            if count > cutoff:
                input_vocab.add_token(word)
        return cls(input_vocab, class_vocab)
    
    @classmethod
    def from_serializable(cls, contents):
        """Intantiate a ReviewVectorizer from a serializable dictionary
        Args:
            contents (dict): the serializable dictionary
        Returns:
            an instance of the ReviewVectorizer class
        """
        review_vocab = Vocabulary.from_serializable(contents['input_vocab'])
        rating_vocab = Vocabulary.from_serializable(contents['class_vocab'])
        return cls(input_vocab=input_vocab, class_vocab=class_vocab)
    
    def to_serializable(self):
        """Create the serializable dictionary for caching
        Returns:
            contents (dict): the serializable dictionary
        """
        return {'input_vocab': self.input_vocab.to_serializable(),
                'class_vocab': self.class_vocab.to_serializable()}

In [52]:
def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will
    ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, 
                            shuffle=shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [53]:
class JobDescriptionClassifier(nn.Module):
    """ a simple perceptron-based classifier """
    def __init__(self, num_features):
        """
        Args:
            num_features (int): the size of the input feature vector
        """
        super(JobDescriptionClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, out_features=1)
    
    def forward(self, x_in, apply_sigmoid=False):
        """The forward pass of the classifier
        Args:
            x_in (torch.Tensor): an input data tensor 
                    x_in.shape should be (batch, num_features)
            apply_sigmoid (bool): a flag for the sigmoid activation
                    should be false if used with the cross-entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch,).
        """
        y_out = self.fc1(x_in).squeeze()
        
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)
        return y_out

In [68]:
from argparse import Namespace
args = Namespace(
    # Data and path information
    frequency_cutoff=25,
    model_state_file='task1-model.pth',
    input_name='task1',
    save_dir='model_storage/task1/',
    vectorizer_file='vectorizer.json',
    # No model hyperparameters
    # Training hyperparameters
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.01,
    num_epochs=100,
    seed=1337,
    # Runtime options
    cuda=True,
    device='cuda',
)

In [69]:
import torch
import torch.optim as optim
import pandas as pd

def make_train_state(args):
    return {'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': 1,
            'test_acc': 1}

train_state = make_train_state(args)
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")

# dataset and vectorizer
dataset = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.input_name)
vectorizer = dataset.get_vectorizer()
# model
classifier = JobDescriptionClassifier(num_features=len(vectorizer.input_vocab))
classifier = classifier.to(args.device)
# loss and optimizer
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)

In [70]:
def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()#.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [71]:
for epoch_index in range(args.num_epochs):
    train_state['epoch_index'] = epoch_index
    # Iterate over training dataset
    # setup: batch generator, set loss and acc to 0, set train mode on
    dataset.set_split('train')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.0
    running_acc = 0.0
    classifier.train()
    for batch_index, batch_dict in enumerate(batch_generator):
        # the training routine is 5 steps:
        # step 1. zero the gradients
        optimizer.zero_grad()
        # step 2. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 3. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch-running_loss) / (batch_index + 1)
        # step 4. use loss to produce gradients
        loss.backward()
        # step 5. use optimizer to take gradient step
        optimizer.step()
        # compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)

    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)

    # Iterate over val dataset
    # setup: batch generator, set loss and acc to 0, set eval mode on
    dataset.set_split('val')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):
        # step 1. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 2. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch - running_loss) / (batch_index + 1)
        # step 3. compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)
    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)

In [72]:
dataset.set_split('test')
batch_generator = generate_batches(dataset,batch_size=args.batch_size,device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()
for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred = classifier(x_in=batch_dict['x_data'].float())
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_batch = loss.item()
    running_loss += (loss_batch - running_loss) / (batch_index + 1)
    # compute the accuracy
    acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_batch - running_acc) / (batch_index + 1)
train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [73]:
print("Test loss: {:.3f}".format(train_state['test_loss']))
print("Test Accuracy: {:.2f}".format(train_state['test_acc']))

Test loss: 0.691
Test Accuracy: 52.52


# Binary Classification

## Data Loading

In [2]:
train, test, val = load_data('task_1')

## Models

### Feed Forward Neural Network

In [3]:
train

,Unnamed: 0,job_description,is_fulltime,tfidf10
0,13999,"['due', 'to', 'continued', 'growth', 'our', 'c...",False,['myob' 'becoming' 'adobe' 'numeracy' 'mackay'...
1,11062,"['seeking', 'an', 'administrator', 'in', 'herv...",False,['administration' 'adaptability' 'areas' 'deme...
2,793,"['qualified', 'panel', 'beater', 'or', 'experi...",True,['air' 'allocated' 'panel' 'dismantler' 'mon' ...
3,16510,"['exciting', 'opportunity', 'for', 'an', 'expe...",False,['migration' 'to' 'planning' 'with' 'workplace...
4,769,"['about', 'mercedes', 'benz', 'adelaide', 'mer...",True,['and' 'of' 'cmv' 'art' 'automotive' 'dealersh...
...,...,...,...,...
12406,11811,"['are', 'you', 'an', 'experienced', 'specialis...",False,['reservoir' 'gynaecologist' 'obstetrician' 'a...
12407,1115,"['flick', 'anticimex', 'is', 'one', 'of', 'aus...",True,['australia' 'although' 'hygiene' 'the' 'crede...
12408,10492,"['restaurant', 'in', 'armadale', 'is', 'seekin...",False,['should' 'least' 'casual' 'at' 'rights' 'cafe...
12409,1298,"['our', 'company', 'is', 'market', 'leader', '...",True,['press' 'in' 'trade' 'drills' 'wire' 'toolmak...


### CNN Conv1D Model

### CNN Conv1D (full job description)

## Comparison / Discussion

# Multi-class Classification

## Models

### Vanilla RNN

### LSTM

## Comparison / Discussion